# Library

In [10]:
import requests
from bs4 import BeautifulSoup as bs

import time, os, random
import pandas as pd
import numpy as np

from selenium import webdriver
import chromedriver_autoinstaller as ca

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## chrome driver 설치

In [2]:
# USB error 메세지 발생 해결을 위한 코드
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [3]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'131'

In [4]:
# # 크롬 드라이버 확인 및 설치(처음 한번만 실행)
# ca.install(True)

# 페이지 접속

In [ ]:
# url = 'https://section.blog.naver.com/BlogHome.naver?directoryNo=0&currentPage=1&groupId=0' # 네이버 블로그 홈

## requests 테스트
- 페이지 접속 가능 여부확인
    - 가능할 경우 출력 : <Response [200]>

In [7]:
# req = requests.get(url)
# print(req)

# 한글 깨짐 해결 코드
# # html = req.content.decode('utf-8') # 한글 깨짐 해결
# # soup = bs(html, 'html.parser')

# soup = bs(req.text, 'html.parser')
# soup.title.text

<Response [200]>


'네이버 블로그'

In [48]:
# request로 수집 에러
keyword = '아이폰 16' # 검색어
page_num = 1 # 페이지 번호
keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=ALL&orderBy=sim&keyword={keyword}'
req = requests.get(keyword_url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'})
print(req)
soup = bs(req.text, 'html.parser')
soup.title.text

<Response [200]>


'네이버 블로그'

In [50]:
# area_list_search = soup.select_one('div.area_list_search')
# list_search_post = area_list_search.select('div.list_search_post')
# list_search_post[0]

AttributeError: 'NoneType' object has no attribute 'select'

## keyword 입력

In [125]:
keyword = '아이폰 16' # 검색어
page_num = 1 # 페이지 번호
tab_option = '글'
if tab_option == '글':
    keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=ALL&orderBy=sim&keyword={keyword}'
elif tab_option == '블로그':
    keyword_url = f'https://section.blog.naver.com/Search/Blog.naver?pageNo={page_num}&orderBy=sim&keyword={keyword}'

## 탭 선택(글, 블로그)

In [ ]:
# tab_option = '블로그'

# if tab_option == '글':
#     page = driver.page_source
#     soup = bs(page, 'html.parser')
#     print('글')
#     print(soup.title.text)

# elif tab_option == '블로그':
#     blog_tab_xpath = '//*[@id="content"]/section/div[1]/div[1]/a[2]'
#     blog_tab = driver.find_element(By.XPATH, blog_tab_xpath)
#     blog_tab.click()
#     driver.implicitly_wait(10)

#     page = driver.page_source
#     soup = bs(page, 'html.parser')
#     print('블로그')
#     print(soup.title.text)

## selenium 작동

In [8]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

In [126]:
driver.get(keyword_url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
# driver.maximize_window() # 브라우져 창 최대화

In [ ]:
# keyword = '아이폰 16' # 검색어
# keyword_input_xpath = '//*[@id="header"]/div[1]/div/div[2]/form/fieldset/div/input' # 검색창 xpath
# keyword_input_box = driver.find_element(By.XPATH, keyword_input_xpath)
# keyword_input_box.send_keys(keyword)
# time.sleep(random.uniform(1, 2))
# keyword_input_box.send_keys(Keys.ENTER)
# driver.implicitly_wait(10)

# 정보 수집

### 🔧 글 선택
- 참고 링크 : https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=ALL&orderBy=sim&keyword={keyword}
- 참고 링크 : https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate=2025-01-03&endDate=2025-01-10&keyword=아이폰 16
- rangeType 옵션(기간)
    - 기간전체 : &rangeType=ALL
    - 최근 1주 : &rangeType=WEEK
    - 최근 1개월 : &rangeType=MONTH
    - 기간 입력 : &startDate=YYYY-mm-dd&endDate=YYYY-mm-dd

- orderBy 옵션(관련도순, 최신순)
    - 관련도순 : &orderBy=sim
    - 최신순 : &orderBy=recentdate

In [127]:
page = driver.page_source
soup = bs(page, 'html.parser')
print(soup.title.text)

네이버 블로그


In [128]:
area_list_search = soup.select_one('div.area_list_search')
list_search_post = area_list_search.select('div.list_search_post')
list_search_post[0]

<div anchor-text="content" bg-move-scroll-on-pop-state="" class="list_search_post" ng-repeat="post in postSearchCtrl.resultList">
<div bg-article-multi-img-style="" class="item multi_pic">
<div class="info_post" ng-class="{only_post: post.thumbnails.length == 0}">
<div class="desc">
<a bg-nclick="srs*l.text" class="desc_inner" href="https://blog.naver.com/cjfgns2/223720560043" ng-href="https://blog.naver.com/cjfgns2/223720560043" target="_blank">
<strong class="title_post">
<!-- ngIf: post.titleHeadColor == '#27C400' -->
<!-- ngIf: post.titleHeadColor == '#FF6565' -->
<span class="title" ng-bind-html="post.title"><strong class="search_keyword">아이폰16</strong> 프로 디자인 보조금 꿀팁까지</span>
</strong>
</a><!-- ngIf: post.contents --><a bg-nclick="srs*l.text" class="text" href="https://blog.naver.com/cjfgns2/223720560043" ng-bind-html="post.contents" ng-href="https://blog.naver.com/cjfgns2/223720560043" ng-if="post.contents" target="_blank">오늘은 <strong class="search_keyword">아이폰16</strong> 프로 디자인과

In [64]:
# 글 제목
post_num = 0
title = list_search_post[post_num].select_one('span.title').text
title

'아이폰16 프로 디자인 보조금 꿀팁까지'

In [65]:
# 블로그 내용
text = list_search_post[post_num].select_one('a.text').text
text

'오늘은 아이폰16 프로 디자인과 달라진 주요 스펙을 정리해 보고, 가격 할인을 받을 수 있는 방법까지 총정리해 보는 시간을 가져볼게요. 서두는 이만하고 바로 본론으로 넘어가 자세히 알아보도록 하겠습니다. <목차> 1.아이폰16 프로 스펙 및 디자인 2.아이폰16 프로 컬러 및 가격 3. 할인 크게... '

In [66]:
# 글 작성자
name_author = list_search_post[post_num].select_one('em.name_author').text
name_author

'얌못'

In [67]:
# 블로그 이름
name_blog = list_search_post[post_num].select_one('span.name_blog').text
name_blog

'경제꿈나무'

In [68]:
# 글 작성 날짜
date = list_search_post[post_num].select_one('span.date').text
date

'17시간 전'

In [69]:
# 글 링크
desc_inner = list_search_post[post_num].select_one('a.desc_inner')['href']
desc_inner

'https://blog.naver.com/cjfgns2/223720560043'

In [70]:
list_search_post[1]

<div anchor-text="content" bg-move-scroll-on-pop-state="" class="list_search_post" ng-repeat="post in postSearchCtrl.resultList">
<div bg-article-multi-img-style="" class="item multi_pic">
<div class="info_post" ng-class="{only_post: post.thumbnails.length == 0}">
<div class="desc">
<a bg-nclick="srs*l.text" class="desc_inner" href="https://blog.naver.com/sexyleep/223712531240" ng-href="https://blog.naver.com/sexyleep/223712531240" target="_blank">
<strong class="title_post">
<!-- ngIf: post.titleHeadColor == '#27C400' -->
<!-- ngIf: post.titleHeadColor == '#FF6565' -->
<span class="title" ng-bind-html="post.title"><strong class="search_keyword">아이폰16</strong>프로 256 가격 성지 할인</span>
</strong>
</a><!-- ngIf: post.contents --><a bg-nclick="srs*l.text" class="text" href="https://blog.naver.com/sexyleep/223712531240" ng-bind-html="post.contents" ng-href="https://blog.naver.com/sexyleep/223712531240" ng-if="post.contents" target="_blank">최신 스마트폰 중에서도 <strong class="search_keyword">아이폰16</str

In [71]:
# 글 제목
post_num = 1
title = list_search_post[post_num].select_one('span.title').text
title

'아이폰16프로 256 가격 성지 할인'

In [72]:
# 블로그 내용
text = list_search_post[post_num].select_one('a.text').text
text

'최신 스마트폰 중에서도 아이폰16 프로는 강력한 성능과 세련된 디자인으로 많은 이들의 관심을 받고 있습니다. 하지만 높은 출고가로 인해 구매를 고민하는 경우가 많습니다. 이번 글에서는 아이폰16 프로의 주요 스펙과 함께 256GB 모델을 더 저렴하게 구매할 수 있는 다양한 방법을... '

In [73]:
# 글 작성자
name_author = list_search_post[post_num].select_one('em.name_author').text
name_author

'테크매니아'

In [74]:
# 블로그 이름
name_blog = list_search_post[post_num].select_one('span.name_blog').text
name_blog

'테크톡톡'

In [75]:
# 글 작성 날짜
date = list_search_post[post_num].select_one('span.date').text
date

'2025. 1. 3.'

In [ ]:
# 글 링크
text_link = list_search_post[post_num].select_one('a.desc_inner')['href']
text_link

'https://blog.naver.com/sexyleep/223712531240'

In [129]:
area_list_search = soup.select_one('div.area_list_search')
list_search_post = area_list_search.select('div.list_search_post')

title_list = []
text_list = []
name_author_list = []
name_blog_list = []
date_list = []
text_link_list = []

for post in list_search_post:
    title = post.select_one('span.title').text
    text = post.select_one('a.text').text
    name_author = post.select_one('em.name_author').text
    name_blog = post.select_one('span.name_blog').text
    date = post.select_one('span.date').text
    text_link = post.select_one('a.desc_inner')['href']
    # print(title, text, name_author, name_blog, text_link)

    title_list.append(title)
    text_list.append(text)
    name_author_list.append(name_author)
    name_blog_list.append(name_blog)
    date_list.append(date)
    text_link_list.append(text_link)
print(len(title_list), len(text_list), len(name_author_list), len(name_blog_list), len(date_list), len(text_link_list))

7 7 7 7 7 7


In [ ]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

In [134]:
keyword = '아이폰 16' # 검색어
tab_option = '글'

title_list = []
text_list = []
name_author_list = []
name_blog_list = []
date_list = []
post_link_list = []

for page_num in range(1, 10):
    if tab_option == '글':
        keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=ALL&orderBy=sim&keyword={keyword}'
    elif tab_option == '블로그':
        keyword_url = f'https://section.blog.naver.com/Search/Blog.naver?pageNo={page_num}&orderBy=sim&keyword={keyword}'

    driver.get(keyword_url)
    driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
    time.sleep(random.uniform(1, 3))

    page = driver.page_source
    soup = bs(page, 'html.parser')
    # print(soup.title.text)
    
    area_list_search = soup.select_one('div.area_list_search')
    list_search_post = area_list_search.select('div.list_search_post')

    for post in list_search_post:
        title = post.select_one('span.title').text
        text = post.select_one('a.text').text
        name_author = post.select_one('em.name_author').text
        name_blog = post.select_one('span.name_blog').text
        date = post.select_one('span.date').text
        post_link = post.select_one('a.desc_inner')['href']

        title_list.append(title)
        text_list.append(text)
        name_author_list.append(name_author)
        name_blog_list.append(name_blog)
        date_list.append(date)
        post_link_list.append(post_link)

print(len(title_list), len(text_list), len(name_author_list), len(name_blog_list), len(date_list), len(post_link_list))

네이버 블로그
네이버 블로그
네이버 블로그
네이버 블로그
네이버 블로그
네이버 블로그
네이버 블로그
네이버 블로그
네이버 블로그
63 63 63 63 63 63


In [135]:
post_dict = {
    'title': title_list,
    'text': text_list,
    'name_author': name_author_list,
    'name_blog': name_blog_list,
    'date': date_list,
    'post_link': post_link_list
}
post_df = pd.DataFrame(post_dict)
post_df

,title,text,name_author,name_blog,date,post_link
0,아이폰16 프로 디자인 보조금 꿀팁까지,"오늘은 아이폰16 프로 디자인과 달라진 주요 스펙을 정리해 보고, 가격 할인을 받을...",얌못,경제꿈나무,17시간 전,https://blog.naver.com/cjfgns2/223720560043
1,아이폰16프로 256 가격 성지 할인,최신 스마트폰 중에서도 아이폰16 프로는 강력한 성능과 세련된 디자인으로 많은 이들...,테크매니아,테크톡톡,2025. 1. 3.,https://blog.naver.com/sexyleep/223712531240
2,아이폰16 비교 성지 보조금 정보까지,이번 아이폰16 시리즈 쫙 비교하고 저는 프로를 선택했어요:) 일반형이랑 많이 고민...,곰순이,곰순이,2025. 1. 3.,https://blog.naver.com/mlifeso/223713468628
3,아이폰16 일반 구매 가이드 나알지,"아이폰16 시리즈가 드디어 공개되며, 많은 분들이 새 스마트폰 구매를 고민하고 계실...",리,이야기,2024. 12. 26.,https://blog.naver.com/ssg2070kk/223706335892
4,아이폰 16프로 가격 비교 최대 지원금 성지 보조금 체크,목차 아이폰 16프로 스펙 공시지원금 선택약정 구매비교 자급제 vs 성지구매 비교 ...,냠냐미,"그때, 그 순간의 기록 ෆ",2024. 12. 30.,https://blog.naver.com/bsh_2310/223709206124
...,...,...,...,...,...,...
58,아이폰16 비교 기본 스펙 싸게사는법 정리,"이번 글에서는 아이폰16 시리즈의 모델별 특징과 출고가를 비교하고, 비용 부담을 덜...",DevelopPharmacy,DevelopPharmacy,2024. 12. 26.,https://blog.naver.com/pangyolee/223706071730
59,아이폰16 프로 블랙 싸게사는법 추천 팁,아이폰16 프로는 최신 기술력과 감각적인 디자인으로 출시 직후부터 큰 관심을 받고 ...,테크매니아,테크톡톡,2025. 1. 1.,https://blog.naver.com/sexyleep/223710511737
60,아이폰16 블루 컬러와 성지 활용 실속 구매 가이드,아이폰16 블루는 전작인 아이폰 15 블루보다 더 진한 색감과 세련된 디자인으로 많...,DevelopPharmacy,DevelopPharmacy,2025. 1. 4.,https://blog.naver.com/pangyolee/223714772074
61,아이폰16 프로 스펙,아이폰16 프로 스펙 2. 아이폰 16 프로 가격 정보 3. 아이폰 16 프로 할인...,DevelopPharmacy,DevelopPharmacy,2025. 1. 5.,https://blog.naver.com/pangyolee/223715710529


In [138]:
new = post_df.drop_duplicates()
new.shape

(63, 6)

## 🔧 블로그 선택
- 참고 링크 : https://section.blog.naver.com/Search/Blog.naver?pageNo=1&orderBy=sim&keyword=아이폰 16
- 참고 링크 : https://section.blog.naver.com/Search/Blog.naver?pageNo=1&orderBy=recentdate&keyword=아이폰 16
- orderBy 옵션
    - 관련도순 : &orderBy=sim
    - 최신순 : &orderBy=recentdate

In [82]:
page_num = 1 # 페이지 번호
tab_option = '블로그'
if tab_option == '글':
    keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=ALL&orderBy=sim&keyword={keyword}'
elif tab_option == '블로그':
    keyword_url = f'https://section.blog.naver.com/Search/Blog.naver?pageNo={page_num}&orderBy=sim&keyword={keyword}'

In [83]:
keyword_url

'https://section.blog.naver.com/Search/Blog.naver?pageNo=1&orderBy=sim&keyword=아이폰 16'

In [84]:
driver.get(keyword_url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
# driver.maximize_window() # 브라우져 창 최대화

In [85]:
page = driver.page_source
soup = bs(page, 'html.parser')
print(soup.title.text)

네이버 블로그


In [87]:
area_list_search = soup.select_one('div.area_list_search')
list_search_blog = area_list_search.select('div.list_search_blog')
list_search_blog[0]

<div anchor-text="content" bg-move-scroll-on-pop-state="" class="list_search_blog" ng-repeat="blog in blogSearchCtrl.resultList">
<div class="item">
<a bg-nclick="srb*l.blogger" class="name_blog" href="https://blog.naver.com/qhrejddl7373" ng-href="https://blog.naver.com/qhrejddl7373" target="_blank">
<em class="text_blog" ng-bind-html="blog.blogName"><strong class="search_keyword">아이폰16</strong>프로사전예약혜택성지모아카페</em>
</a>
<p class="blog_intro" ng-bind-html="blog.blogDesc"><strong class="search_keyword">아이폰16</strong>프로사전예약혜택성지모아카페 정보 공유 블로그 입니다~</p>
<div class="writer_info">
<a bg-nclick="srb*l.blogger" class="author" href="https://blog.naver.com/qhrejddl7373" ng-href="https://blog.naver.com/qhrejddl7373" target="_blank">
<div class="thumbnail_author">
<img alt="블로거 썸네일" bg-image="https://blogpfthumb-phinf.pstatic.net/MjAyNDA5MTNfMTc3/MDAxNzI2MTk2NTg4Njgx.uAZNONyJWghf7L_Anmni-CY7bw7n1TYuESshHirSrCcg.d0j2fA62wXmE-SqYrDRIv206ty97Ju_CFz2HLa6oafEg.PNG/m2.png/m2.png?type=s1" class="img_author"

In [99]:
# 블로그 제목
post_num = 0
text_blog = list_search_blog[post_num].select_one('em.text_blog').text
text_blog

'아이폰16프로사전예약혜택성지모아카페'

In [100]:
# 블로그 내용
blog_intro = list_search_blog[post_num].select_one('p.blog_intro').text
blog_intro

'아이폰16프로사전예약혜택성지모아카페 정보 공유 블로그 입니다~'

In [101]:
# 글 작성자
name_author = list_search_blog[post_num].select_one('em.name_author').text
name_author

'성지모아카페'

In [120]:
# 블로그 링크
blog_link = list_search_blog[post_num].select_one('a.name_blog')['href']
blog_link

'https://blog.naver.com/crnzlwbrc'

In [103]:
# 블로그 제목
post_num = 1
text_blog = list_search_blog[post_num].select_one('em.text_blog').text
text_blog

'아이폰16'

In [108]:
# 블로그 내용
blog_intro = list_search_blog[post_num].select_one('p.blog_intro').text
blog_intro

''

In [116]:
list_search_blog[post_num].select_one('p.blog_intro').text == ''

True

In [105]:
# 글 작성자
name_author = list_search_blog[post_num].select_one('em.name_author').text
name_author

'아이폰16'

In [119]:
# 블로그 링크
blog_link = list_search_blog[post_num].select_one('a.name_blog')['href']
blog_link

'https://blog.naver.com/crnzlwbrc'

In [121]:
# 정보 수집
text_blog_list = []
blog_intro_list = []
name_author_list = []
blog_link_list = []
for blog in list_search_blog:
    text_blog = blog.select_one('em.text_blog').text
    if blog.select_one('p.blog_intro').text == '':
        blog_intro = np.nan
    else:
        blog_intro = blog.select_one('p.blog_intro').text
    name_author = blog.select_one('em.name_author').text
    blog_link = blog.select_one('a.name_blog')['href']
    # print(text_blog, blog_intro, name_author, blog_link)

    text_blog_list.append(text_blog)
    blog_intro_list.append(blog_intro)
    name_author_list.append(name_author)
    blog_link_list.append(blog_link)

print(len(text_blog_list), len(blog_intro_list), len(name_author_list), len(name_blog_list))

10 10 10 10


In [123]:
blog_dict = {
    'text_blog': text_blog_list,
    'blog_intro': blog_intro_list,
    'name_author': name_author_list,
    'blog_link': blog_link_list
}
blog_df = pd.DataFrame(blog_dict)
blog_df

,text_blog,blog_intro,name_author,blog_link
0,아이폰16프로사전예약혜택성지모아카페,아이폰16프로사전예약혜택성지모아카페 정보 공유 블로그 입니다~,성지모아카페,https://blog.naver.com/qhrejddl7373
1,아이폰16,NaN,아이폰16,https://blog.naver.com/crnzlwbrc
2,"먹고, 쉬고.","맛집, 카페, 여행 [촬영 : 아이폰 16프로]",잠만보,https://blog.naver.com/unicodin
3,먹짱일기장,아이폰 16 pro max오너의 맛도리 블로그,유댕,https://blog.naver.com/eliyoujin
4,KT플라자 광화문중앙,아이폰16 즉시개통,KT광화문중앙직영점,https://blog.naver.com/bcbc233
5,냠냠 미식노트,구미지역의 맛집 탐방✨ [아이폰16 촬영],냠냠로드,https://blog.naver.com/alswn120679
6,푸딩이와 삐약이의 여행일기S2,아이폰 pro max 15 / 아이폰 16 사용,나는야 이푸딩,https://blog.naver.com/kjjxv9309
7,post_uooooodi,"아이폰16pro 나의앨범 86,029장",uooooodi,https://blog.naver.com/uooooodi
8,KT대화역점,친절상담KT대화점아이폰16사전예약중,kkkyou1111,https://blog.naver.com/kkkyou1111
9,Mlog,서울 지역의 맛집과 카페 꽃집을 리뷰해요 아이폰 16pro로 촬영해요,종이,https://blog.naver.com/fate081000


# 🔧 종합
- 글, 블로그 옵션 설정 적용 : 관련도, 기간 등

In [ ]:
keyword = '아이폰 16' # 검색어
tab_option = '글'

In [ ]:
if tab_option == '글':
    title_list = []
    text_list = []
    name_author_list = []
    name_blog_list = []
    date_list = []
    post_link_list = []
    for page_num in range(1, 10):
        keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=ALL&orderBy=sim&keyword={keyword}'

        driver.get(keyword_url)
        driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
        time.sleep(random.uniform(1, 3))

        page = driver.page_source
        soup = bs(page, 'html.parser')
        # print(soup.title.text)
        
        area_list_search = soup.select_one('div.area_list_search')
        list_search_post = area_list_search.select('div.list_search_post')

        for post in list_search_post:
            title = post.select_one('span.title').text
            text = post.select_one('a.text').text
            name_author = post.select_one('em.name_author').text
            name_blog = post.select_one('span.name_blog').text
            date = post.select_one('span.date').text
            post_link = post.select_one('a.desc_inner')['href']

            title_list.append(title)
            text_list.append(text)
            name_author_list.append(name_author)
            name_blog_list.append(name_blog)
            date_list.append(date)
            post_link_list.append(post_link)

    print(len(title_list), len(text_list), len(name_author_list), len(name_blog_list), len(date_list), len(post_link_list))
    
elif tab_option == '블로그':
    for page_num in range(1, 10):
        keyword_url = f'https://section.blog.naver.com/Search/Blog.naver?pageNo={page_num}&orderBy=sim&keyword={keyword}'

        driver.get(keyword_url)
        driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
        time.sleep(random.uniform(1, 3))

        page = driver.page_source
        soup = bs(page, 'html.parser')
        # print(soup.title.text)

        area_list_search = soup.select_one('div.area_list_search')
        list_search_post = area_list_search.select('div.list_search_post')

        for post in list_search_post:
            title = post.select_one('span.title').text
            text = post.select_one('a.text').text
            name_author = post.select_one('em.name_author').text
            name_blog = post.select_one('span.name_blog').text
            date = post.select_one('span.date').text
            post_link = post.select_one('a.desc_inner')['href']

            title_list.append(title)
            text_list.append(text)
            name_author_list.append(name_author)
            name_blog_list.append(name_blog)
            date_list.append(date)
            post_link_list.append(post_link)

    print(len(title_list), len(text_list), len(name_author_list), len(name_blog_list), len(date_list), len(post_link_list))

In [139]:
driver.quit()

# END